# Project 1 Notebook
## Project members: Andrew, Tim, Mide

## Feel free to change/add cells as needed
## If you are going to make a function try to let each other know that you made a function so we can all use it. (Just incase)

## Points to hit:
* Point 1: Correlation between highest grossing movies and critic/user ratings
* Point 2: Correlation between the rating of a movie and the total gross
* Point 3: Correlation between movie genres and total gross
---

## Import needed packages for project
---

In [5]:
import pandas as pd
import numpy as np
import sqlite3
import zipfile
import matplotlib.pyplot
%matplotlib inline

# Data cleaning work here

## IMDB Database
---

#### Extract the database

In [6]:
with zipfile.ZipFile('data/im.db.zip', 'r') as imdbzip:
    imdbzip.extractall('data/')

#### Connect to the database

In [7]:
con = sqlite3.connect('data/im.db')

#### Perform the Query to see 

In [8]:
imdb_df = pd.read_sql(
"""
SELECT
 movie_id,
 primary_title as title,
 averagerating,
 numvotes,
 genres,
 runtime_minutes

FROM 
 movie_ratings
 INNER JOIN movie_basics USING (movie_id)
 JOIN movie_akas USING (movie_id)
 
WHERE
 (runtime_minutes IS NOT NULL) AND (region = 'US')  AND (numvotes > 200000)

GROUP BY movie_id

ORDER BY numvotes DESC

""", con)

<img src='data/IMDBDF.jpg'>

## Movies foreign and domestic gross earnings
---

In [9]:
movie_gross = pd.read_csv('data/bom.movie_gross.csv')
movie_gross.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [10]:
movie_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


## Rotten Tomatoes movie information
---

In [11]:
rt_movie_info = pd.read_csv('data/rt.movie_info.tsv', sep='\t')
rt_movie_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [12]:
rt_movie_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


## Rotten Tomatoes movie reviews
---

In [13]:
rt_movie_reviews = pd.read_csv('data/rt.reviews.tsv', sep='\t', encoding='latin-1')
rt_movie_reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [14]:
rt_movie_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


## The Movies Database
---

In [15]:
tmdb_movies = pd.read_csv('data/tmdb.movies.csv', index_col=0)
tmdb_movies.head()

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [16]:
tmdb_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26517 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          26517 non-null  object 
 1   id                 26517 non-null  int64  
 2   original_language  26517 non-null  object 
 3   original_title     26517 non-null  object 
 4   popularity         26517 non-null  float64
 5   release_date       26517 non-null  object 
 6   title              26517 non-null  object 
 7   vote_average       26517 non-null  float64
 8   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 2.0+ MB


## The Numbers (budget and gross)
---

In [17]:
tn_movie_budgets = pd.read_csv('data/tn.movie_budgets.csv')
tn_movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [18]:
tn_movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


## Selecting Dataset for point 1
---

In [20]:
#merge imdb and movie_gross dataframes
tdf = imdb_df.merge(movie_gross)

In [21]:
#selected needed columns into new df
gross_title = pd.DataFrame(tdf, columns=['title', 'domestic_gross','foreign_gross'])

#creating a df sorted by domestic gross
top10_dgross = gross_title.sort_values('domestic_gross', ascending=False, ignore_index=True)[:10]
top10_dgross['domestic_gross'] = top10_dgross['domestic_gross'].apply(lambda x: x/1000000)

top10_dgross

,title,domestic_gross,foreign_gross
0,Black Panther,700.1,646900000
1,Avengers: Infinity War,678.8,"1,369.5"
2,Jurassic World,652.3,"1,019.4"
3,Star Wars: The Last Jedi,620.2,712400000
4,Incredibles 2,608.6,634200000
5,Rogue One: A Star Wars Story,532.2,523900000
6,Finding Dory,486.3,542300000
7,Avengers: Age of Ultron,459.0,946400000
8,The Dark Knight Rises,448.1,636800000
9,The Hunger Games: Catching Fire,424.7,440300000


In [47]:
# tdf[['title','averagerating','numvotes','domestic_gross','foreign_gross']]

,title,averagerating,numvotes,domestic_gross,foreign_gross
0,Inception,8.8,1841066,292600000.0,535700000
1,The Dark Knight Rises,8.4,1387769,448100000.0,636800000
2,Interstellar,8.6,1299334,188000000.0,489400000
3,Django Unchained,8.4,1211405,162800000.0,262600000
4,The Wolf of Wall Street,8.2,1035358,116900000.0,275100000
...,...,...,...,...,...
250,Sully,7.5,202718,125100000.0,115700000
251,The Adventures of Tintin,7.3,202430,77600000.0,296400000
252,War for the Planet of the Apes,7.5,201943,146900000.0,343800000
253,Contagion,6.6,201536,75700000.0,59800000


## Selected Dataset for point 2
---

## Selecting Dataset for point 3
---